In [21]:
from catboost import CatBoostClassifier, Pool
from catboost.utils import eval_metric
from sklearn.datasets import make_multilabel_classification
from sklearn.model_selection import train_test_split
import pandas as pd
import sklearn.metrics as metrics
from sklearn.metrics import accuracy_score, classification_report
import numpy as np
import plotly.express as px
import seaborn as sns
from IPython import display
import matplotlib.pyplot as plt
import awswrangler as wr
from sklearn.metrics import (
    accuracy_score,
    confusion_matrix,
    classification_report,
    multilabel_confusion_matrix,
)
from catboost.utils import select_threshold
from skmultilearn.model_selection.iterative_stratification import (
    IterativeStratification,
)
from sklearn.model_selection import StratifiedKFold, KFold
from tqdm import tqdm
from sklearn.utils import class_weight


In [22]:

wr.config.s3_endpoint_url = "http://192.168.1.4:8333"

multilabelDf = wr.s3.read_parquet(
    path=f"s3://multilabel_df/",
    dataset=True,
)
allVisionFeatures = [x for x in multilabelDf.columns if "vision_" in x]
caseFeatures = [
    "Circumstances_of_Accident",
    "Model",
    # # "Vehicle_Still_Driveable",
    # # "NCB_Stat",
    # # "Assembly_Type",
    # # "Claim_Type",
    "Vehicle_Type",
    # # "Sum_Insured",
    # "Repairer",
    # # # "Repairer_Apprv_Count",
    # "Collision_With",
    # "Handling_Insurer",
]
allInputFeature = caseFeatures + allVisionFeatures

targetCol = [
   'bonnet', 'bumper_front', 'grille', 'fog_lamp_rh', 'headlamp_lh', 'headlamp_rh', 'door_front_lh', 'door_front_rh', 'air_conditioning', 'cooling_fan', 'radiator', 'bumper_rear', 'misc', 'engine', 'airbag', 'front_panel', 'undercarriage', 'wheel', 'fender_front_lh', 'fender_front_rh', 'door_mirror_rh', 'rear_quarter_lh', 'interior', 'tail_lamp_lh', 'fog_lamp_lh', 'tail_lamp_rh', 'windscreen_front', 'rear_compartment', 'rear_panel', 'rear_quarter_rh', 'door_rear_rh', 'door_mirror_lh', 'door_rear_lh', 'windscreen_rear'
]
allPredModel = []
allPreds = []
allGt = []


In [23]:

for part in tqdm(targetCol):
    X = multilabelDf[allInputFeature + ["CaseID"]]
    Y = multilabelDf[part].to_frame()
    mulitlearnStratify = StratifiedKFold(n_splits=2)
    allCaseIdByPart = []
    allPredByPart = []
    allGtByPart = []

    for kfoldId, (train_index, test_index) in enumerate(mulitlearnStratify.split(X, Y)):
        X_train, X_test = X.iloc[train_index], X.iloc[test_index]
        Y_train, Y_test = Y.iloc[train_index], Y.iloc[test_index]
        trainCaseId = X_train["CaseID"]
        testCaseId = X_test["CaseID"]
        X_train = X_train[allInputFeature]
        X_test = X_test[allInputFeature]
        clsWeight=class_weight.compute_class_weight(class_weight='balanced',classes=np.unique(Y_train.values),y=Y_train.values.squeeze(1))

        testClsWeights = {
            0 : clsWeight[0],
            1 : clsWeight[1]

        }
        sampleWeights = class_weight.compute_sample_weight(class_weight=testClsWeights, y=Y_test.values)

        train_pool = Pool(
            X_train, Y_train, cat_features=caseFeatures + allVisionFeatures
        )
        test_pool = Pool(X_test, Y_test, cat_features=caseFeatures + allVisionFeatures)
        clf = CatBoostClassifier(
            loss_function="Logloss",
            eval_metric="F1",
            iterations=200,
            early_stopping_rounds=100,
            task_type="GPU",
            use_best_model=True,
            class_weights= clsWeight,

            # class_names=["not_dmg", "dmg"],
        )
        clf.fit(
            train_pool, eval_set=test_pool, metric_period=100, plot=False, verbose=1
        )
        test_predict = clf.predict(X_test)
        b = Y_test.values.squeeze(1).astype(np.int64)
        acc = accuracy_score(b, test_predict, sample_weight=sampleWeights)
        confMat = confusion_matrix(b, test_predict, sample_weight=sampleWeights)
        pos_count = len(Y_test[Y_test[part] == 1]) / len(Y_test)
        tn = confMat[0][0]
        tp = confMat[1][1]
        fp = confMat[0][1]
        fn = confMat[1][0]
        totalSample = fp + fn + tp + tn
        acc = (tp + tn) / (fp + fn + tp + tn)
        precision = tp / (tp + fp)
        recall = tp / (tp + fn)
        # assert (tp / (tp + fn)) + (fn / (tp + fn)) == (tn / (tn + fp)) + (
        #     fp / (tn + fp)
        # ) 
        allPredByPart.extend(test_predict)
        
        allCaseIdByPart.extend(testCaseId)
        allGtByPart.extend(b)

        allPredModel.append(
            {   "kfold" : kfoldId,
                "part": part,
                "tp": tp / (tp + fn),
                "tn": tn / (tn + fp),
                "fp": fp / (tn + fp),
                "fn": fn / (tp + fn),
                "acc": acc,
                "precision": precision,
                "recall": recall,
                "f1": (2 * precision * recall) / (precision + recall),
                "pos_count": pos_count,
            }
        )
    allPreds.append({
    "CaseID": allCaseIdByPart,
    part : allPredByPart})
    allGt.append({
    "CaseID": allCaseIdByPart,
    part : allGtByPart})
 

evalMetrics = pd.json_normalize(allPredModel)
avgPrecision = evalMetrics["precision"].mean()
avgRecall = evalMetrics["recall"].mean()
avgF1 = evalMetrics["f1"].mean()
avgTp = evalMetrics["tp"].mean()
avgTn = evalMetrics["tn"].mean()
avgAcc = evalMetrics["acc"].mean()
avgFn = evalMetrics["fn"].mean()

print(f"Avg Precision : {avgPrecision}")
print(f"Avg Recall : {avgRecall}")
print(f"Avg F1 : {avgF1}")
print(f"Avg TP : {avgTp}")
print(f"Avg TN : {avgTn}")
print(f"Avg FN : {avgFn}")

print(f"avgAccs : {avgAcc}")

  0%|          | 0/34 [00:00<?, ?it/s]

Learning rate set to 0.08543
0:	learn: 1.0000000	test: 1.0000000	best: 1.0000000 (0)	total: 14.8ms	remaining: 2.95s
100:	learn: 1.0000000	test: 1.0000000	best: 1.0000000 (0)	total: 1.9s	remaining: 1.86s
bestTest = 1
bestIteration = 0
Shrink model to first 1 iterations.
Learning rate set to 0.08543
0:	learn: 1.0000000	test: 1.0000000	best: 1.0000000 (0)	total: 14.1ms	remaining: 2.8s
100:	learn: 1.0000000	test: 1.0000000	best: 1.0000000 (0)	total: 1.94s	remaining: 1.91s
bestTest = 1
bestIteration = 0
Shrink model to first 1 iterations.


  3%|▎         | 1/34 [00:09<05:23,  9.80s/it]

Learning rate set to 0.08543
0:	learn: 1.0000000	test: 1.0000000	best: 1.0000000 (0)	total: 43.2ms	remaining: 8.6s
100:	learn: 1.0000000	test: 1.0000000	best: 1.0000000 (0)	total: 1.9s	remaining: 1.86s
bestTest = 1
bestIteration = 0
Shrink model to first 1 iterations.
Learning rate set to 0.08543
0:	learn: 1.0000000	test: 1.0000000	best: 1.0000000 (0)	total: 12.9ms	remaining: 2.56s
100:	learn: 1.0000000	test: 1.0000000	best: 1.0000000 (0)	total: 1.56s	remaining: 1.53s
bestTest = 1
bestIteration = 0
Shrink model to first 1 iterations.


  6%|▌         | 2/34 [00:18<04:50,  9.07s/it]

...

Learning rate set to 0.08543
0:	learn: 1.0000000	test: 1.0000000	best: 1.0000000 (0)	total: 20.1ms	remaining: 4.01s
100:	learn: 1.0000000	test: 1.0000000	best: 1.0000000 (0)	total: 2.09s	remaining: 2.05s
bestTest = 1
bestIteration = 0
Shrink model to first 1 iterations.
Learning rate set to 0.08543
0:	learn: 1.0000000	test: 1.0000000	best: 1.0000000 (0)	total: 19.8ms	remaining: 3.94s
100:	learn: 1.0000000	test: 1.0000000	best: 1.0000000 (0)	total: 2.21s	remaining: 2.16s
bestTest = 1
bestIteration = 0
Shrink model to first 1 iterations.


 97%|█████████▋| 33/34 [05:57<00:11, 11.47s/it]

Learning rate set to 0.08543
0:	learn: 1.0000000	test: 1.0000000	best: 1.0000000 (0)	total: 18.7ms	remaining: 3.72s
100:	learn: 1.0000000	test: 1.0000000	best: 1.0000000 (0)	total: 2.06s	remaining: 2.02s
bestTest = 1
bestIteration = 0
Shrink model to first 1 iterations.
Learning rate set to 0.08543
0:	learn: 1.0000000	test: 1.0000000	best: 1.0000000 (0)	total: 16.8ms	remaining: 3.35s
100:	learn: 1.0000000	test: 1.0000000	best: 1.0000000 (0)	total: 1.87s	remaining: 1.83s
bestTest = 1
bestIteration = 0
Shrink model to first 1 iterations.


100%|██████████| 34/34 [06:07<00:00, 10.80s/it]

Avg Precision : 0.9534222905793529
Avg Recall : 0.8999453517273395
Avg F1 : 0.8991221906095671
Avg TP : 0.8999453517273395
Avg TN : 0.9399091530779394
Avg FN : 0.1000546482726605
avgAccs : 0.9199274082932373


In [24]:
evalMetrics = evalMetrics[evalMetrics["kfold"] == 0]

In [25]:
fig = px.bar(evalMetrics, x = 'part', y = 'tp')

fig.add_hline(y=avgTp, line_width=2, line_color="red")

fig.update_layout(
    yaxis = dict(
        tickmode = 'array',
        tickvals = [np.format_float_positional(avgTp, 2), 0.2, 0.5, 0.7],
    )
)
fig

In [26]:
fig = px.bar(evalMetrics, x = 'part', y = 'tn')

fig.add_hline(y=avgTn, line_width=2, line_color="red")

fig.update_layout(
    yaxis = dict(
        tickmode = 'array',
        tickvals = [np.format_float_positional(avgTn, 2), 0.2, 0.5, 0.7],
    )
)
fig

In [27]:
fig = px.bar(evalMetrics, x = 'part', y = 'acc')

fig.add_hline(y=avgTn, line_width=2, line_color="red")

fig.update_layout(
    yaxis = dict(
        tickmode = 'array',
        tickvals = [np.format_float_positional(avgAcc, 2), 0.2, 0.5, 0.7],
    )
)

In [33]:
fig = px.bar(evalMetrics, x = 'part', y = 'f1')

fig.add_hline(y=avgF1, line_width=2, line_color="red")

fig.update_layout(
    yaxis = dict(
        tickmode = 'array',
        tickvals = [np.format_float_positional(avgAcc, 2), 0.2, 0.5, 0.7],
    )
)

In [34]:
multilabelPred = pd.DataFrame(pd.DataFrame(allPreds[0]))
for p in tqdm(allPreds[1:]):
    dfPart = pd.DataFrame(p)
    multilabelPred = multilabelPred.merge(dfPart, on="CaseID")
multilabelPred.sort_values(by="CaseID", inplace=True)
multilabelPred

100%|██████████| 33/33 [00:59<00:00,  1.82s/it]


,CaseID,bonnet,bumper_front,grille,fog_lamp_rh,headlamp_lh,headlamp_rh,door_front_lh,door_front_rh,air_conditioning,...,fog_lamp_lh,tail_lamp_rh,windscreen_front,rear_compartment,rear_panel,rear_quarter_rh,door_rear_rh,door_mirror_lh,door_rear_lh,windscreen_rear
143200,10000004,0,1,0,0,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0
465095,10000012,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
52001,10000016,1,1,1,1,1,1,0,0,1,...,1,0,1,0,0,0,0,0,0,0
143201,10000021,0,1,0,0,0,0,0,0,1,...,0,0,0,0,0,0,0,1,1,0
536990,10000030,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
208650,13563056,0,1,1,0,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0
503810,13563063,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
208651,13563071,0,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
503811,13563089,0,0,0,0,0,0,0,0,0,...,0,1,0,1,0,0,0,0,0,1


In [35]:
multilabelGt = pd.DataFrame(pd.DataFrame(allGt[0]))
for p in tqdm(allGt[1:]):
    dfPart = pd.DataFrame(p)
    multilabelGt = multilabelGt.merge(dfPart, on="CaseID")
multilabelGt.sort_values(by="CaseID", inplace=True)


100%|██████████| 33/33 [00:59<00:00,  1.82s/it]


In [36]:
multilabelGt = multilabelGt[multilabelPred.columns]
multilabelGt

,CaseID,bonnet,bumper_front,grille,fog_lamp_rh,headlamp_lh,headlamp_rh,door_front_lh,door_front_rh,air_conditioning,...,fog_lamp_lh,tail_lamp_rh,windscreen_front,rear_compartment,rear_panel,rear_quarter_rh,door_rear_rh,door_mirror_lh,door_rear_lh,windscreen_rear
143200,10000004,0,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
465095,10000012,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
52001,10000016,1,1,1,1,1,1,0,0,1,...,0,0,1,0,0,0,0,0,0,0
143201,10000021,0,1,0,0,1,0,0,0,0,...,1,0,0,0,0,0,0,0,1,0
536990,10000030,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
208650,13563056,0,1,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
503810,13563063,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
208651,13563071,0,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
503811,13563089,0,0,0,0,0,0,0,0,0,...,0,1,0,1,0,0,0,0,0,1


In [37]:
def hamming_score(y_true, y_pred):
    acc_list = []
    for i in range(y_true.shape[0]):
        set_true = set( np.where(y_true[i])[0] )
        set_pred = set( np.where(y_pred[i])[0] )
        #print('\nset_true: {0}'.format(set_true))
        #print('set_pred: {0}'.format(set_pred))
        tmp_a = None
        if len(set_true) == 0 and len(set_pred) == 0:
            tmp_a = 1
        else:
            tmp_a = len(set_true.intersection(set_pred))/\
                    float( len(set_true) )
        #print('tmp_a: {0}'.format(tmp_a))
        acc_list.append(tmp_a)
    return np.mean(acc_list), acc_list

In [38]:
subsetAcc2, acc_list = hamming_score(multilabelGt[targetCol].values, multilabelPred[targetCol].values)
subsetAcc2

0.8821735761393544